## Through topic modelling, we want to find out the common topics derived from people's positive sentiments regarding public spaces in Paya Lebar

In [1]:
# Standard dataframe packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Text analytics packages
import gensim
import nltk
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
#Note: Try either of the following encodings: encoding='utf-8' or encoding='ISO-8859-1'
#import data
data=pd.read_excel('Sentiment.xlsx',sheet_name="Main",skipfooter=1)

In [3]:
#quick peek at the data
data.head()

,Social Offerings,Openness,Amenities,Cultural Heritage,Full text
0,We are staying in Geylang near the river. We g...,Very nice. So far I haven't encountered bad pe...,I think it’s already nice just the mosquitoes....,Usually we will go to the mall. I don’t get to...,We are staying in Geylang near the river. We g...
1,Not so really bonded to community. Paya Lebar ...,Last time my impression was this place was for...,The most modern building will be Paya Lebar Qu...,Yeah then the URA got a certain development pl...,Not so really bonded to community. Paya Lebar ...
2,OK actually we go to the shopping mall. This i...,Oh yeah absolutely absolutely welcome people i...,Let's say I was very surprised by PLQ. I think...,"Absolutely important, yes that's why I think P...",OK actually we go to the shopping mall. This i...
3,I started liking this place after 4 months sta...,"Around the condo. Just like family, we know ea...","Playground one, two things very boring. I thin...",Don't really know any cultural heritage here. ...,I started liking this place after 4 months sta...
4,My house is near to church. I’m a christian so...,I feel connected because mostly food. I usuall...,The river reminds me of my hometown.,No I don’t go there.,My house is near to church. I’m a christian so...


In [4]:
data2=data.loc[:,"Full text"].astype("str")

In [5]:
#import external packages for sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [8]:
# nltk.download('vader_lexicon')

In [9]:
analyzer = SentimentIntensityAnalyzer()
#quick peek at data generated from vader
# for i in range(len(data2)):
#     scores = analyzer.polarity_scores(data2.iloc[i])
#     print(scores)

In [10]:
# VADER produces four sentiment metrics from these word ratings The first three, positive, neutral and negative,
# represent the proportion of the text that falls into those categories. 
# The polarity_scores gives us numerical values for use of negative, neutral, and positive word choice. 
# The compound value reflects the overall sentiment normalized to range of -1 being very negative to +1 being very positive.
#Printing the sentiment nicely in a table format. 
my_vader_score_compound = [ ] 
my_vader_score_positive = [ ] 
my_vader_score_negative = [ ] 
my_vader_score_neutral = [ ] 

for i in range(len(data2)):
    my_analyzer = analyzer.polarity_scores(data2.iloc[i])
    my_vader_score_compound.append(my_analyzer['compound'])
    my_vader_score_positive.append(my_analyzer['pos'])
    my_vader_score_negative.append(my_analyzer['neg']) 
    my_vader_score_neutral.append(my_analyzer['neu']) 


#converting sentiment values to numpy for easier usage
my_vader_score_compound = np.array(my_vader_score_compound)
my_vader_score_positive = np.array(my_vader_score_positive)
my_vader_score_negative = np.array(my_vader_score_negative)
my_vader_score_neutral = np.array(my_vader_score_neutral)

In [11]:
dataVader=data2.copy()
dataVader=pd.DataFrame(dataVader)

In [12]:
dataVader['Score'] = my_vader_score_compound
dataVader['postve'] = my_vader_score_positive
dataVader['neg'] = my_vader_score_negative
dataVader['neu'] = my_vader_score_neutral
dataVader.head()

,Full text,Score,postve,neg,neu
0,We are staying in Geylang near the river. We g...,0.9668,0.184,0.000,0.816
1,Not so really bonded to community. Paya Lebar ...,0.9886,0.110,0.036,0.855
2,OK actually we go to the shopping mall. This i...,0.9996,0.240,0.012,0.748
3,I started liking this place after 4 months sta...,0.9914,0.205,0.022,0.773
4,My house is near to church. I’m a christian so...,0.9420,0.190,0.037,0.773


In [14]:
#Setting threshold for what comment is considered positive sentiment
pos_data=dataVader[dataVader["Score"]>=0.75]
pos_text=pos_data["Full text"]

In [15]:
#create stop words list
stop_words = nltk.corpus.stopwords.words('english')
stop_words.extend(['quite', 'paya', 'lebar', 'PLQ', 'would','go','come',
                   'already','feel','like','place','around','area','this',
                   'know','also','really','nice','enough','good','yeah','important','love','absolutely','still','okay','even','though','cuz'
                  ,'make'])

#create tokenizer
#\w matches [a-zA-Z0-9_]. This tokenizer splits the string using regular expressions. E.g - '[A-Z]\w+' will select just the capital words
wtk = nltk.tokenize.RegexpTokenizer(r'\w+')

#create lemmatizer
wnl = nltk.stem.wordnet.WordNetLemmatizer()

In [16]:
#create bigrams
from nltk.util import bigrams
def process_bigrams(documents):
    process_docs = []
    doc_tokens = []
    for doc in documents:
        doc = doc.lower()
        doc_tokens = wtk.tokenize(doc)
        doc_tokens = [token for token in doc_tokens if token not in stop_words]
        doc_tokens = [wnl.lemmatize(token) for token in doc_tokens] #Can try spacy or see how to use the (nltk.pos_tag()) with lemmatizer
        doc_tokens= ["_".join(t) for t in bigrams(doc_tokens)]
        print(doc_tokens)
        doc_tokens = [token for token in doc_tokens if len(token) > 2]
        process_docs.append(doc_tokens)
    return process_docs
process_pos_bigrams=process_bigrams(pos_text)

['staying_geylang', 'geylang_near', 'near_river', 'river_square', 'square_child', 'child_taking', 'taking_care', 'care_taking', 'taking_ballet', 'ballet_class', 'class_playground', 'playground_accessible', 'accessible_u', 'u_every', 'every_time', 'time_usually', 'usually_take', 'take_20', '20_minute', 'minute_house', 'house_walk', 'walk_playground', 'playground_accessibility', 'accessibility_store', 'store_plq', 'plq_many', 'many_amenity', 'amenity_far', 'far_encountered', 'encountered_bad', 'bad_people', 'people_singaporean', 'singaporean_friendly', 'friendly_think', 'think_mosquito', 'mosquito_far', 'far_nothing', 'nothing_complaint', 'complaint_usually', 'usually_mall', 'mall_get', 'get_see', 'see_heritage', 'heritage_want', 'want_see', 'see_way']
['bonded_community', 'community_commercial', 'commercial_talk', 'talk_living', 'living_estate', 'estate_value', 'value_three', 'three_lot', 'lot_light', 'light_industrial', 'industrial_behind', 'behind_car', 'car_servicing', 'servicing_kin

In [17]:
# Create a dictionary representation of the documents. Each word gets an id
dictionary = gensim.corpora.Dictionary(process_pos_bigrams)
#filtering out bigrams based on their occurrence in the whole sentiments dataset
dictionary.filter_extremes(no_below=1, no_above=0.2)

In [18]:
# Transforming corpus into bag of words vectors
corpus_vect = [dictionary.doc2bow(text) for text in process_pos_bigrams]

In [19]:
#Train lda model
num_topics = 4
ldamodel = gensim.models.ldamodel.LdaModel(corpus_vect, num_topics = num_topics, id2word=dictionary, passes=25, random_state= 0)


In [20]:
#formatting the topics in print
for num ,topic in ldamodel.show_topics(formatted=True, num_topics=num_topics, num_words=7):
    print(str(num)+": "+ topic)

0: 0.003*"public_space" + 0.003*"social_offering" + 0.003*"participate_community" + 0.002*"spend_time" + 0.002*"community_activity" + 0.002*"shopping_centre" + 0.001*"community_friendly"
1: 0.005*"city_plaza" + 0.003*"meet_friend" + 0.002*"hangout_indonesian" + 0.002*"favourite_food" + 0.002*"katong_complex" + 0.002*"east_coast" + 0.002*"indonesian_food"
2: 0.003*"open_space" + 0.003*"shopping_mall" + 0.002*"serai_market" + 0.002*"geylang_serai" + 0.002*"lot_food" + 0.002*"kampong_life" + 0.002*"food_court"
3: 0.004*"shopping_mall" + 0.003*"hawker_centre" + 0.002*"city_plaza" + 0.002*"geylang_serai" + 0.002*"table_chair" + 0.002*"green_space" + 0.002*"community_event"


In [21]:
#Evaluate the performance of trained lda model 
#perplexity score the lower the better
log_perplexity = ldamodel.log_perplexity(corpus_vect)
perplexity = 2**(-log_perplexity)
print('Perplexity: ',perplexity)

# Compute Coherence Score
#coherence score the higher the better
from gensim.models import CoherenceModel
coherence_model_lda = CoherenceModel(model=ldamodel, texts=process_pos_bigrams, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

Perplexity:  337.88939639354726

Coherence Score:  0.46483439596217224
